In [19]:
from tqdm import tqdm
import numpy as np
import random
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import string
from torch.utils.data import Dataset, DataLoader
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
nltk.download('stopwords')
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
! unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [22]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(42)
MAX_LENGTH =80
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [23]:
train= pd.read_csv('../input/nlp-getting-started/train.csv')
test= pd.read_csv("../input/nlp-getting-started/test.csv")

In [24]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

def clean_sentence(sentence):
    # remove URLS 
    sentence = re.sub(r'http\S+', ' ', sentence)
    # remove emoji's
    sentence = remove_emoji(sentence)
    # remove punctuation
    sentence = re.sub("[^0-9A-Za-z ]", "" , sentence)
    # remove double spaces
    sentence = sentence.replace('  ',"")
    
    return sentence.strip()

def remove_stopwords(tokens, stopwords):
    clean_token = [word for word in tokens if word not in stopwords]
    return clean_token

# lemmalization 
def lemmatize(tokens, lemma):
    lemmatized_tokens = [lemma.lemmatize(token, pos = 'v') for token in tokens]
    return lemmatized_tokens

In [25]:
stopwords = nltk.corpus.stopwords.words('english')
lemma = WordNetLemmatizer()

def processing(df, stopwords, lemma):
    df['text'] = df['text'].apply(lambda sentence: sentence.lower())
    df['text'] = df['text'].apply(lambda sentence: clean_sentence(sentence))
    
    
processing(train, stopwords, lemma)
processing(test, stopwords, lemma)

In [26]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,all residents asked to shelter in place are be...,1
3,6,NaN,NaN,13000 people receive wildfires evacuation orde...,1
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1


In [27]:
from transformers import BertTokenizer
token = BertTokenizer.from_pretrained('bert-large-uncased')

In [28]:
from transformers import BertModel

class TweetDataset(Dataset):
    def __init__(self, df):
        self.x = df['text']
        self.y = df['target']
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        sentence = self.x[idx]
        return sentence, self.y[idx]
    
train_dataset = TweetDataset(train)

In [29]:
def collate_fn(data):
    sentences = []
    labels = []
    for item in data:
        sentences.append(item[0])
        labels.append(item[1])
    
    data = token.batch_encode_plus(batch_text_or_text_pairs=sentences,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=80,
                                   return_tensors='pt')
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)
    labels = torch.LongTensor(labels).to(device)
    return input_ids, attention_mask, token_type_ids, labels

In [30]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=32,
                          collate_fn=collate_fn,
                          shuffle=True,
                          drop_last=True)

In [37]:
from transformers import BertModel
pretrained = BertModel.from_pretrained('bert-large-uncased')
pretrained.to(device)

for param in pretrained.parameters():
    param.requires_grad_(False)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
class bert_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(1024, 2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids).last_hidden_state[:, 0]
        # out = self.dropout(out)
        out = self.fc(out).softmax(dim=1)
        return out

In [39]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    prediction = preds.argmax(dim=1)
    correct = (prediction == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [40]:
model = bert_model()
model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.0005)

In [41]:
epoch_loss = 0
epoch_acc = 0
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids, attention_mask, token_type_ids, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss
        
        acc = binary_accuracy(outputs, labels)
        epoch_acc += acc.item()
    
    print("The training loss at epoch {} is {}; The training accuracy is {}".format(epoch, epoch_loss / len(train_loader), 
                                                                                    round(epoch_acc/len(train_loader), 3)))
    epoch_loss = 0
    epoch_acc = 0

100%|██████████| 237/237 [01:02<00:00,  3.78it/s]


The training loss at epoch 0 is 0.5720729827880859; The training accuracy is 0.748


100%|██████████| 237/237 [01:02<00:00,  3.78it/s]


The training loss at epoch 1 is 0.5280571579933167; The training accuracy is 0.789


100%|██████████| 237/237 [01:02<00:00,  3.78it/s]


...

The training loss at epoch 7 is 0.49499326944351196; The training accuracy is 0.816


100%|██████████| 237/237 [01:02<00:00,  3.77it/s]


The training loss at epoch 8 is 0.49276334047317505; The training accuracy is 0.818


100%|██████████| 237/237 [01:02<00:00,  3.77it/s]

The training loss at epoch 9 is 0.49039047956466675; The training accuracy is 0.82


In [ ]:
test_set: 0.8

# prediction

In [42]:
data = token.batch_encode_plus(batch_text_or_text_pairs=test['text'], truncation=True, padding='max_length', max_length=30, return_tensors='pt')
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
token_type_ids = data['token_type_ids'].to(device)
prediction = model(input_ids, attention_mask, token_type_ids).argmax(dim=1).cpu()

In [43]:
test['target'] = prediction
submission = test[['id', 'target']]
submission.to_csv('submission2.csv', index=False)